# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Project Setup

In [1]:
# all import statements needed for the project, for example:

import os
import re
from bs4 import BeautifulSoup
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
import sqlalchemy as db

In [2]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TLC_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = "taxi_zones"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
WEATHER_CSV_DIR = "weather_data"

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [3]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Load Taxi Zones
In this section, we loaded the taxi shapefile that corresponds location IDs to geographical latitudes and logitudes. 
* The `load_taxi_zones` function reads the shapefile and use GeoPandas to read the file
* The `lookup_coords_for_taxi_zone_id` function takes location IDs and the loaded shapefile and returns a tuple of latitude and logitude
* The `make_loc_id_coords_dict` creates a dictionary of location IDs and coordinates that will be used in data cleaning

In [4]:
def load_taxi_zones(shapefile):
    taxi_zones = gpd.read_file(shapefile)
    return taxi_zones

In [5]:
taxi_zones = load_taxi_zones(TAXI_ZONES_SHAPEFILE)
taxi_zones.crs

<Projected CRS: EPSG:2263>
Name: NAD83 / New York Long Island (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - New York - counties of Bronx; Kings; Nassau; New York; Queens; Richmond; Suffolk.
- bounds: (-74.26, 40.47, -71.8, 41.3)
Coordinate Operation:
- name: SPCS83 New York Long Island zone (US survey foot)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [6]:
def lookup_coords_for_taxi_zone_id(zone_loc_id, loaded_taxi_zones=taxi_zones):
    zone = loaded_taxi_zones[loaded_taxi_zones["LocationID"] == zone_loc_id]
    centroid = zone.geometry.centroid.iloc[0]
    centroid_geo = gpd.GeoSeries([centroid], crs=loaded_taxi_zones.crs).to_crs(epsg=CRS).iloc[0]

    latitude = centroid_geo.y
    longitude = centroid_geo.x

    return (latitude, longitude)

In [7]:
def make_loc_id_coords_dict(loaded_taxi_zones):
    id_coords_dict = {}
    for loc_id in loaded_taxi_zones["LocationID"]:
        id_coords_dict[loc_id] = lookup_coords_for_taxi_zone_id(loc_id, loaded_taxi_zones)

    return id_coords_dict

In [8]:
ID_COORDS_DICT = make_loc_id_coords_dict(taxi_zones)

### Calculate Sample Size

In [9]:
def calculate_sample_size(population):
    confidence_level = 0.95
    margin_of_error = 0.05
    proportion = 0.5
    
    from scipy.stats import norm

    z_score = norm.ppf(1 - (1 - confidence_level) / 2)

    # Cochran’s
    n = (z_score**2 * proportion * (1 - proportion)) / (margin_of_error**2)
    
    # Adjust for finite population
    n_adj = n / (1 + (n - 1) / population)
    
    return int(round(n_adj)) 

### Common Functions
* `get_all_urls_from_taxi_page` fetches information on the taxi page and finds all "Yellow Taxi Trip Records" urls and "High Volume For-Hire Vehicle Trip Records" urls
* `find_parquet_urls` uses regex to filter the urls that ends with ".parquet" to make sure that the urls are parquet files
* `download_parquet` creates a directory and downloads relevant parquets into the directory
* `get_and_clean_month` filters the urls that are from January 2020 to August 2024
* `sample_monthly` function reads all the parquet files in a directory, finds the file with largest number of rows and computes the sample size using the "maximum population". Next it creates samples for all files using the computed sample size and combine them into a single dataframe.

In [10]:
def get_all_urls_from_taxi_page(taxi_page):
    response = requests.get(taxi_page)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    yellow_tags = soup.find_all("a", attrs={"title": "Yellow Taxi Trip Records"})
    fhvhv_tags = soup.find_all("a", attrs={"title": "High Volume For-Hire Vehicle Trip Records"})

    yellow_urls = [a["href"].strip() for a in yellow_tags]
    fhvhv_urls = [a["href"].strip() for a in fhvhv_tags]
    
    return yellow_urls, fhvhv_urls

In [11]:
def find_parquet_urls(urls):
    pattern = re.compile(r"\.parquet$")
    parquet_urls = [url for url in urls if pattern.search(url)]    
    return parquet_urls

In [12]:
def download_parquet(urls, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for url in urls:
        filename = os.path.basename(url)
        output_path = os.path.join(output_dir, filename)
        if os.path.exists(output_path):
            continue        
        response = requests.get(url, stream=True)
        with open(output_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024): 
                if chunk:
                    f.write(chunk)
        print(f"Downloaded {filename} to {output_dir}")

In [13]:
def get_and_clean_month(urls):
    pattern = re.compile(r"(202[0-3]-(0[1-9]|1[0-2])|2024-(0[1-8]))")
    cleaned_urls = [url for url in urls if pattern.search(url)]
    return cleaned_urls

In [14]:
def read_parquet_to_df(directory):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(".parquet")]
    all_dataframe = []
    for file in files:
        
        df = pd.read_parquet(file)
        all_dataframe.append(df)
    if all_dataframe:
        combined_df = pd.concat(all_dataframe, ignore_index=True)
        return combined_df

In [15]:
def clean_parquet_column(file_path, columns_to_keep):
    df = pd.read_parquet(file_path)
    cleaned_df = df[columns_to_keep]
    return cleaned_df

In [16]:
yellow_urls, fhvhv_urls = get_all_urls_from_taxi_page(TLC_URL)
taxi_parquet = find_parquet_urls(yellow_urls)
uber_parquet = find_parquet_urls(fhvhv_urls)
taxi_urls = get_and_clean_month(taxi_parquet)
uber_urls = get_and_clean_month(uber_parquet)

In [17]:
def sample_monthly(directory):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.parquet')]
    
    max_rows = 0
    for file in files:
        df = pd.read_parquet(file)
        max_rows = max(max_rows, len(df))

    sample_size = calculate_sample_size(max_rows)
    print(f"Sample size for all months: {sample_size}")

    sampled_dataframes = []
    for file in files:
        df = pd.read_parquet(file)
        sampled_df = df.sample(n=sample_size, random_state=30, replace=False)
        sampled_dataframes.append(sampled_df)
    print("Finished sampling")

    if sampled_dataframes:
        combined_sampled_df = pd.concat(sampled_dataframes, ignore_index=True)
        return combined_sampled_df

### Process Taxi Data

In [18]:
taxi_data_dir = "taxi_data"
download_parquet(taxi_urls, taxi_data_dir)

In [19]:
sampled_taxi_df = sample_monthly(taxi_data_dir)

Sample size for all months: 384
Finished sampling


/var/folders/rm/llw7py457rl4zlvsw5ymljw80000gn/T/ipykernel_49567/1458695063.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_sampled_df = pd.concat(sampled_dataframes, ignore_index=True)


In [20]:
sampled_taxi_df.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,airport_fee
0,2,2023-06-05 06:50:05,2023-06-05 07:11:09,3.0,5.55,1.0,N,132,197,1,27.5,0.0,0.5,3.25,0.00,1.0,34.00,0.0,1.75,NaN
1,2,2023-06-12 11:11:39,2023-06-12 11:20:59,1.0,1.71,1.0,N,239,48,2,12.1,0.0,0.5,0.00,0.00,1.0,16.10,2.5,0.00,NaN
2,2,2023-06-07 18:27:39,2023-06-07 18:46:53,1.0,2.67,1.0,N,234,231,1,19.1,2.5,0.5,5.12,0.00,1.0,30.72,2.5,0.00,NaN
3,1,2023-06-29 21:50:08,2023-06-29 22:04:14,2.0,1.10,1.0,N,186,230,1,13.5,3.5,0.5,3.70,0.00,1.0,22.20,2.5,0.00,NaN
4,1,2023-06-10 22:25:59,2023-06-10 22:36:26,1.0,1.20,1.0,N,246,249,1,10.7,3.5,0.5,3.10,0.00,1.0,18.80,2.5,0.00,NaN
5,2,2023-06-20 08:31:24,2023-06-20 09:22:59,1.0,11.09,1.0,N,138,143,1,52.0,5.0,0.5,10.40,6.55,1.0,79.70,2.5,1.75,NaN
6,1,2023-06-27 08:40:33,2023-06-27 08:47:33,1.0,1.20,1.0,N,236,151,1,7.9,2.5,0.5,2.40,0.00,1.0,14.30,2.5,0.00,NaN
7,2,2023-06-20 14:08:07,2023-06-20 14:21:00,1.0,1.58,1.0,N,79,233,1,12.8,0.0,0.5,3.36,0.00,1.0,20.16,2.5,0.00,NaN
8,2,2023-06-22 18:17:30,2023-06-22 18:35:05,5.0,0.86,1.0,N,48,230,2,15.6,2.5,0.5,0.00,0.00,1.0,22.10,2.5,0.00,NaN
9,2,2023-06-22 14:13:38,2023-06-22 14:27:20,1.0,3.99,1.0,N,144,88,1,20.5,0.0,0.5,2.00,0.00,1.0,26.50,2.5,0.00,NaN


In [38]:
def get_and_clean_taxidata(dataframe):
    try: 
        print(f"Cleaning the sample dataframe...")

        if not isinstance(dataframe, pd.DataFrame):
            raise ValueError("must Pandas DataFrame")

        # look up the latitude and longitude (get those coordinates)
        dataframe[["latitude_pickup", "longitude_pickup"]] = dataframe["PULocationID"].map(ID_COORDS_DICT).apply(pd.Series)
        dataframe[["latitude_dropoff", "longitude_dropoff"]] = dataframe["DOLocationID"].map(ID_COORDS_DICT).apply(pd.Series)

        # remove some location IDs not valid and distance is 0
        dataframe = dataframe.dropna(subset=['latitude_pickup', 'longitude_pickup', 'latitude_dropoff', 'longitude_dropoff'])
        dataframe = dataframe[dataframe["trip_distance"] != 0]

        # airport_fee combine with Airport_fee
        dataframe["airport_fee"] = dataframe["Airport_fee"].combine_first(dataframe["airport_fee"])
        dataframe.drop(columns=["Airport_fee"], inplace=True)
        dataframe["airport_fee"] = dataframe["airport_fee"].fillna(0)
        
        # remove unnecessary columns
        columns_to_keep = [
            'tpep_pickup_datetime', 'tpep_dropoff_datetime',
            'trip_distance', 
            'latitude_pickup', 'longitude_pickup', 'latitude_dropoff', 'longitude_dropoff', 
            'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 
            'improvement_surcharge', 'congestion_surcharge', 'airport_fee', 
            'total_amount'
        ]
        dataframe = dataframe[columns_to_keep]

        # normalize column names
        dataframe.columns = [col.lower().replace(' ', '_') for col in dataframe.columns]

        # normalizing and using appropriate column types for the respective data;
        dataframe['tpep_pickup_datetime'] = pd.to_datetime(dataframe['tpep_pickup_datetime'])
        dataframe['tpep_dropoff_datetime'] = pd.to_datetime(dataframe['tpep_dropoff_datetime'])
        dataframe['trip_distance'] = dataframe['trip_distance'].astype(float)

        # caculate part of total_money 
        dataframe['congestion_surcharge'] = dataframe['congestion_surcharge'].fillna(0)
        dataframe['total_money'] = (dataframe['fare_amount'] + dataframe['extra'] + dataframe['mta_tax'] + 
                                     dataframe['tip_amount'] + dataframe['tolls_amount'] + 
                                     dataframe['improvement_surcharge'] + dataframe['congestion_surcharge'])

        # Add airport_fee to total_money if within airport coordinates
        # dataframe['total_money'] += dataframe.apply(lambda row: row['airport_fee'] if 
        #     (LGA_BOX_COORDS[0][0] <= row['latitude_pickup'] <= LGA_BOX_COORDS[1][0] and 
        #      LGA_BOX_COORDS[0][1] <= row['longitude_pickup'] <= LGA_BOX_COORDS[1][1]) or 
        #     (JFK_BOX_COORDS[0][0] <= row['latitude_pickup'] <= JFK_BOX_COORDS[1][0] and 
        #      JFK_BOX_COORDS[0][1] <= row['longitude_pickup'] <= JFK_BOX_COORDS[1][1]) 
        #     else 0, axis=1)

        # Compare total_money with total_amount
        comparison_result = (dataframe['total_money'] == dataframe['total_amount'])
        print(f"Total money matches total amount: {comparison_result.all()}")

        # for Yellow Taxi data, remove trips that start and/or end outside of (40.560445, -74.242330) and (40.908524, -73.717047) ie NEW_YORK_BOX_COORDS.
        lat_min, lon_min = NEW_YORK_BOX_COORDS[0]
        lat_max, lon_max = NEW_YORK_BOX_COORDS[1]

        dataframe = dataframe[
            (dataframe['latitude_pickup'].between(lat_min, lat_max)) &
            (dataframe['longitude_pickup'].between(lon_min, lon_max)) &
            (dataframe['latitude_dropoff'].between(lat_min, lat_max)) &
            (dataframe['longitude_dropoff'].between(lon_min, lon_max))
        ]

        return dataframe

    except Exception as e:
        print(f"Error processing the dataframe: {e}")
        return None  

In [39]:
taxi_data = get_and_clean_taxidata(sampled_taxi_df)

Cleaning the sample dataframe...
Total money matches total amount: False


In [40]:
taxi_data.head(10)

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,latitude_pickup,longitude_pickup,latitude_dropoff,longitude_dropoff,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,airport_fee,total_amount,total_money
0,2023-06-05 06:50:05,2023-06-05 07:11:09,5.55,40.646985,-73.786530,40.694542,-73.830924,27.5,0.0,0.5,3.25,0.00,1.0,0.0,1.75,34.00,32.25
1,2023-06-12 11:11:39,2023-06-12 11:20:59,1.71,40.783961,-73.978632,40.762253,-73.989845,12.1,0.0,0.5,0.00,0.00,1.0,2.5,0.00,16.10,16.10
2,2023-06-07 18:27:39,2023-06-07 18:46:53,2.67,40.740337,-73.990458,40.717773,-74.007880,19.1,2.5,0.5,5.12,0.00,1.0,2.5,0.00,30.72,30.72
3,2023-06-29 21:50:08,2023-06-29 22:04:14,1.10,40.748497,-73.992438,40.759818,-73.984197,13.5,3.5,0.5,3.70,0.00,1.0,2.5,0.00,22.20,24.70
4,2023-06-10 22:25:59,2023-06-10 22:36:26,1.20,40.753309,-74.004016,40.734576,-74.002875,10.7,3.5,0.5,3.10,0.00,1.0,2.5,0.00,18.80,21.30
5,2023-06-20 08:31:24,2023-06-20 09:22:59,11.09,40.774376,-73.873628,40.775965,-73.987646,52.0,5.0,0.5,10.40,6.55,1.0,2.5,1.75,79.70,77.95
6,2023-06-27 08:40:33,2023-06-27 08:47:33,1.20,40.780436,-73.957012,40.797962,-73.968168,7.9,2.5,0.5,2.40,0.00,1.0,2.5,0.00,14.30,16.80
7,2023-06-20 14:08:07,2023-06-20 14:21:00,1.58,40.727620,-73.985937,40.749914,-73.970443,12.8,0.0,0.5,3.36,0.00,1.0,2.5,0.00,20.16,20.16
8,2023-06-22 18:17:30,2023-06-22 18:35:05,0.86,40.762253,-73.989845,40.759818,-73.984197,15.6,2.5,0.5,0.00,0.00,1.0,2.5,0.00,22.10,22.10
9,2023-06-22 14:13:38,2023-06-22 14:27:20,3.99,40.720889,-73.996919,40.703358,-74.011515,20.5,0.0,0.5,2.00,0.00,1.0,2.5,0.00,26.50,26.50


In [41]:
taxi_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20869 entries, 0 to 21503
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   tpep_pickup_datetime   20869 non-null  datetime64[us]
 1   tpep_dropoff_datetime  20869 non-null  datetime64[us]
 2   trip_distance          20869 non-null  float64       
 3   latitude_pickup        20869 non-null  float64       
 4   longitude_pickup       20869 non-null  float64       
 5   latitude_dropoff       20869 non-null  float64       
 6   longitude_dropoff      20869 non-null  float64       
 7   fare_amount            20869 non-null  float64       
 8   extra                  20869 non-null  float64       
 9   mta_tax                20869 non-null  float64       
 10  tip_amount             20869 non-null  float64       
 11  tolls_amount           20869 non-null  float64       
 12  improvement_surcharge  20869 non-null  float64       
 13  conges

In [42]:
taxi_data.describe()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,latitude_pickup,longitude_pickup,latitude_dropoff,longitude_dropoff,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,airport_fee,total_amount,total_money
count,20869,20869,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000,20869.000000
mean,2022-05-01 23:23:35.520628,2022-05-01 23:40:27.647515,3.313074,40.753511,-73.966552,40.755856,-73.970556,15.666208,1.228502,0.490718,2.726723,0.456978,0.544477,2.183502,0.082922,22.794868,23.297109
min,2020-01-01 00:20:28,2020-01-01 00:33:35,0.010000,40.576961,-74.167234,40.576961,-74.174002,-171.700000,-5.000000,-0.500000,0.000000,-15.380000,-1.000000,-2.500000,-1.750000,-188.080000,-188.080000
25%,2021-02-28 14:22:00,2021-02-28 14:42:00,1.090000,40.740439,-73.989845,40.740337,-73.989845,7.200000,0.000000,0.500000,0.000000,0.000000,0.300000,2.500000,0.000000,12.600000,13.300000
50%,2022-04-30 10:32:36,2022-04-30 10:44:33,1.810000,40.758028,-73.977698,40.758028,-73.977698,10.700000,0.500000,0.500000,2.160000,0.000000,0.300000,2.500000,0.000000,16.800000,17.660000
75%,2023-07-01 09:11:58,2023-07-01 09:26:38,3.400000,40.773633,-73.961764,40.775932,-73.959635,17.500000,2.500000,0.500000,3.500000,0.000000,1.000000,2.500000,0.000000,24.500000,25.200000
max,2024-08-31 23:35:57,2024-08-31 23:44:08,55.600000,40.897932,-73.735554,40.899528,-73.726655,7000.500000,11.750000,0.800000,33.440000,40.000000,1.000000,2.500000,1.750000,7010.850000,7010.850000
std,NaN,NaN,4.135789,0.032516,0.045546,0.033581,0.037030,50.405868,1.513872,0.088052,3.153968,1.878248,0.350941,0.871432,0.349541,51.654328,51.577493


### Processing Uber Data
This section downloads the relevant parquet files from the taxi website and creates a sample according to the sampling function of the Uber data. The sample dataframe is cleaned using the `get_and_clean_uber_data` function.

* The `filter_uber_and_sample_monthly` function uses similar logic as the `sample_monthly` function, but it filters Uber trips before sampling

* reads parquet files in the directory and filter Uber data each month, then creates a sample of each month and integrate the sample datasets into one dataset.

* The `get_and_clean_uber_data` function takes a dataframe and returns a cleaned dataframe that:
    * Filtered Uber rides
    * Converted Location IDs to latitude lognitude coordinates
    * Computed total fares for each ride
    * Filtered rides that start and/or end within the New York bounding box
    * Dropped columns that are irrelevant to later parts of the project
    * Normalized column names and removed invalid data

In [49]:
uber_data_dir = "uber_data"
download_parquet(uber_urls, uber_data_dir)

In [54]:
def filter_uber_and_sample_monthly(directory):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.parquet')]
    
    max_rows = 0
    for file in files:
        df = pd.read_parquet(file, columns=["hvfhs_license_num"])
        filtered_df = df[df["hvfhs_license_num"] == "HV0003"]
        max_rows = max(max_rows, len(filtered_df))
    sample_size = calculate_sample_size(max_rows)
    print(f"Sample size for all months: {sample_size}")
    
    sampled_dataframes = []
    for file in files:
        df = pd.read_parquet(file)
        filtered_df = df[df["hvfhs_license_num"] == "HV0003"]
        sampled_df = filtered_df.sample(n=sample_size, random_state=30, replace=False)
        sampled_dataframes.append(sampled_df)

    if sampled_dataframes:
        combined_sampled_df = pd.concat(sampled_dataframes, ignore_index=True)
        return combined_sampled_df

In [55]:
sampled_uber_df = filter_uber_and_sample_monthly(uber_data_dir)

Sample size for all months: 384


/var/folders/rm/llw7py457rl4zlvsw5ymljw80000gn/T/ipykernel_49567/1403387323.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_sampled_df = pd.concat(sampled_dataframes, ignore_index=True)


In [56]:
sampled_uber_df.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B02765,B02765,2021-03-25 09:31:19,2021-03-25 09:32:12,2021-03-25 09:34:06,2021-03-25 09:39:19,165,165,1.05,...,0.70,0.00,NaN,0.0,6.21,N,N,,N,N
1,HV0003,B02888,B02888,2021-03-24 12:18:27,2021-03-24 12:23:05,2021-03-24 12:24:54,2021-03-24 12:53:16,70,35,11.59,...,2.74,0.00,NaN,0.0,29.45,N,N,,N,N
2,HV0003,B02865,B02865,2021-03-08 12:23:10,2021-03-08 12:25:52,2021-03-08 12:25:52,2021-03-08 12:32:20,262,229,2.19,...,1.24,2.75,NaN,0.0,7.16,N,N,,N,N
3,HV0003,B02875,B02875,2021-03-04 22:14:05,2021-03-04 22:19:17,2021-03-04 22:19:38,2021-03-04 22:25:14,25,25,0.83,...,0.67,0.00,NaN,3.0,9.10,N,N,,N,N
4,HV0003,B02395,B02395,2021-03-16 05:03:32,2021-03-16 05:10:15,2021-03-16 05:12:15,2021-03-16 05:26:09,60,168,2.71,...,0.00,0.00,NaN,0.0,12.94,N,N,,N,N


In [57]:
sampled_uber_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21504 entries, 0 to 21503
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   hvfhs_license_num     21504 non-null  object        
 1   dispatching_base_num  21504 non-null  object        
 2   originating_base_num  21500 non-null  object        
 3   request_datetime      21504 non-null  datetime64[us]
 4   on_scene_datetime     21504 non-null  datetime64[us]
 5   pickup_datetime       21504 non-null  datetime64[us]
 6   dropoff_datetime      21504 non-null  datetime64[us]
 7   PULocationID          21504 non-null  int64         
 8   DOLocationID          21504 non-null  int64         
 9   trip_miles            21504 non-null  float64       
 10  trip_time             21504 non-null  int64         
 11  base_passenger_fare   21504 non-null  float64       
 12  tolls                 21504 non-null  float64       
 13  bcf             

In [58]:
def get_and_clean_uber_data(df):
    fare_columns = ["base_passenger_fare", "tolls", "bcf", "sales_tax", "congestion_surcharge", "airport_fee"]
    columns_to_keep = ["request_datetime", "on_scene_datetime", "pickup_datetime", "dropoff_datetime",
                       "pickup_lat", "pickup_lon", "dropoff_lat", "dropoff_lon", "trip_miles",
                       "total_fare", "tips"]
    # filter uber data
    df = df[df["hvfhs_license_num"] == "HV0003"]

    # convert LocationID to coordinates
    df[["pickup_lat", "pickup_lon"]] = df["PULocationID"].map(ID_COORDS_DICT).apply(pd.Series)
    df[["dropoff_lat", "dropoff_lon"]] = df["DOLocationID"].map(ID_COORDS_DICT).apply(pd.Series)
    
    # remove invalid locations & 0 mile trips
    df = df.dropna(subset=["pickup_lat", "pickup_lon", "dropoff_lat", "dropoff_lon"])
    df = df[df["trip_miles"] != 0]
    
    # filter trips within the bounding box
    ((min_lat, min_lon), (max_lat, max_lon)) = NEW_YORK_BOX_COORDS
    pickup_in_box = (
        (df["pickup_lat"] >= min_lat) & (df["pickup_lat"] <= max_lat) &
        (df["pickup_lon"] >= min_lon) & (df["pickup_lon"] <= max_lon)
    )
    dropoff_in_box = (
        (df["dropoff_lat"] >= min_lat) & (df["dropoff_lat"] <= max_lat) &
        (df["dropoff_lon"] >= min_lon) & (df["dropoff_lon"] <= max_lon)
    )
    df = df[pickup_in_box & dropoff_in_box]
        
    # compute total fare
    df[fare_columns] = df[fare_columns].fillna(0)
    df["total_fare"] = df[fare_columns].sum(axis=1)

    df = df[columns_to_keep]
    
    return df

In [59]:
uber_data = get_and_clean_uber_data(sampled_uber_df)

In [60]:
uber_data.head(10)

,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon,trip_miles,total_fare,tips
0,2021-03-25 09:31:19,2021-03-25 09:32:12,2021-03-25 09:34:06,2021-03-25 09:39:19,40.620924,-73.956824,40.620924,-73.956824,1.05,8.85,0.0
1,2021-03-24 12:18:27,2021-03-24 12:23:05,2021-03-24 12:24:54,2021-03-24 12:53:16,40.763352,-73.868395,40.664003,-73.910258,11.59,34.59,0.0
2,2021-03-08 12:23:10,2021-03-08 12:25:52,2021-03-08 12:25:52,2021-03-08 12:32:20,40.775932,-73.946510,40.756729,-73.965146,2.19,18.40,0.0
3,2021-03-04 22:14:05,2021-03-04 22:19:17,2021-03-04 22:19:38,2021-03-04 22:25:14,40.685634,-73.986114,40.685634,-73.986114,0.83,8.45,3.0
4,2021-03-16 05:03:32,2021-03-16 05:10:15,2021-03-16 05:12:15,2021-03-16 05:26:09,40.833990,-73.885900,40.807347,-73.916822,2.71,16.80,0.0
5,2021-03-04 16:27:53,2021-03-04 16:30:52,2021-03-04 16:32:20,2021-03-04 16:43:20,40.841708,-73.941399,40.837827,-73.926158,2.28,10.83,0.0
6,2021-03-18 20:56:27,2021-03-18 21:00:19,2021-03-18 21:01:06,2021-03-18 21:13:02,40.740337,-73.990458,40.756729,-73.965146,2.54,18.02,0.0
7,2021-03-13 00:35:54,2021-03-13 00:38:16,2021-03-13 00:40:16,2021-03-13 00:49:16,40.640590,-73.976199,40.641886,-74.004653,1.39,6.29,1.0
8,2021-03-17 15:38:12,2021-03-17 15:41:44,2021-03-17 15:43:26,2021-03-17 16:24:06,40.620924,-73.956824,40.766238,-73.995135,12.48,61.31,0.0
9,2021-03-13 12:23:41,2021-03-13 12:26:47,2021-03-13 12:27:54,2021-03-13 12:45:38,40.612218,-73.995259,40.580922,-73.961217,4.08,20.27,3.0


In [61]:
uber_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20650 entries, 0 to 21503
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   request_datetime   20650 non-null  datetime64[us]
 1   on_scene_datetime  20650 non-null  datetime64[us]
 2   pickup_datetime    20650 non-null  datetime64[us]
 3   dropoff_datetime   20650 non-null  datetime64[us]
 4   pickup_lat         20650 non-null  float64       
 5   pickup_lon         20650 non-null  float64       
 6   dropoff_lat        20650 non-null  float64       
 7   dropoff_lon        20650 non-null  float64       
 8   trip_miles         20650 non-null  float64       
 9   total_fare         20650 non-null  float64       
 10  tips               20650 non-null  float64       
dtypes: datetime64[us](4), float64(7)
memory usage: 1.9 MB


In [62]:
uber_data.head()

,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon,trip_miles,total_fare,tips
0,2021-03-25 09:31:19,2021-03-25 09:32:12,2021-03-25 09:34:06,2021-03-25 09:39:19,40.620924,-73.956824,40.620924,-73.956824,1.05,8.85,0.0
1,2021-03-24 12:18:27,2021-03-24 12:23:05,2021-03-24 12:24:54,2021-03-24 12:53:16,40.763352,-73.868395,40.664003,-73.910258,11.59,34.59,0.0
2,2021-03-08 12:23:10,2021-03-08 12:25:52,2021-03-08 12:25:52,2021-03-08 12:32:20,40.775932,-73.946510,40.756729,-73.965146,2.19,18.40,0.0
3,2021-03-04 22:14:05,2021-03-04 22:19:17,2021-03-04 22:19:38,2021-03-04 22:25:14,40.685634,-73.986114,40.685634,-73.986114,0.83,8.45,3.0
4,2021-03-16 05:03:32,2021-03-16 05:10:15,2021-03-16 05:12:15,2021-03-16 05:26:09,40.833990,-73.885900,40.807347,-73.916822,2.71,16.80,0.0


In [63]:
uber_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20650 entries, 0 to 21503
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   request_datetime   20650 non-null  datetime64[us]
 1   on_scene_datetime  20650 non-null  datetime64[us]
 2   pickup_datetime    20650 non-null  datetime64[us]
 3   dropoff_datetime   20650 non-null  datetime64[us]
 4   pickup_lat         20650 non-null  float64       
 5   pickup_lon         20650 non-null  float64       
 6   dropoff_lat        20650 non-null  float64       
 7   dropoff_lon        20650 non-null  float64       
 8   trip_miles         20650 non-null  float64       
 9   total_fare         20650 non-null  float64       
 10  tips               20650 non-null  float64       
dtypes: datetime64[us](4), float64(7)
memory usage: 1.9 MB


In [64]:
uber_data.describe()

,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon,trip_miles,total_fare,tips
count,20650,20650,20650,20650,20650.000000,20650.000000,20650.000000,20650.000000,20650.000000,2.065000e+04,20650.000000
mean,2022-04-29 17:28:44.550799,2022-04-29 17:32:18.193753,2022-04-29 17:33:26.248184,2022-04-29 17:51:28.953462,40.738333,-73.934669,40.737495,-73.934471,4.437571,2.569772e+01,0.815646
min,2020-01-01 00:26:03,2020-01-01 00:35:28,2020-01-01 00:36:58,2020-01-01 00:41:39,40.561994,-74.170885,40.561994,-74.186421,0.020000,-1.110223e-16,0.000000
25%,2021-02-25 21:54:38.500000,2021-02-25 21:56:17.250000,2021-02-25 21:57:11.250000,2021-02-25 22:09:12.750000,40.691507,-73.984197,40.690787,-73.984052,1.540000,1.248000e+01,0.000000
50%,2022-04-27 20:18:27.500000,2022-04-27 20:23:05,2022-04-27 20:23:24,2022-04-27 20:51:21,40.737698,-73.948522,40.737698,-73.947442,2.795000,1.966000e+01,0.000000
75%,2023-06-29 17:47:55,2023-06-29 17:51:19,2023-06-29 17:52:39,2023-06-29 18:02:40.750000,40.775932,-73.899735,40.775932,-73.898957,5.680000,3.202000e+01,0.000000
max,2024-08-31 23:33:56,2024-08-31 23:39:56,2024-08-31 23:41:13,2024-09-01 00:23:56,40.899528,-73.726655,40.899528,-73.726655,50.530000,3.114300e+02,41.360000
std,NaN,NaN,NaN,NaN,0.068433,0.063956,0.068735,0.068116,4.374298,1.976913e+01,2.463201


### Processing Weather Data
In this section, we processed the weather data and creates dataframes with hourly and daily granularity information retaining relevant information only.
*  `get_all_weather_csvs` returns the weather csv files in the directory
*  `clean_month_weather_data_hourly` takes csv files and returns a dataframe that contains hourly precipation and wind speed information
*  `clean_month_weather_data_daily` takes csv files and returns a dataframe that contains daily precipation, wind speed, and snowfall information. The function fills in values according to the data description for better data processing later
*  `load_and_clean_weather_data` concatnates all daily dataframes and all hourly dataframes into two large dataframes that contains all daily weather data and all hourly data respectively.

In [65]:
def get_all_weather_csvs(directory):
    weather_csvs = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.csv')]
    return weather_csvs

In [66]:
def clean_month_weather_data_hourly(csv_file):
    df = pd.read_csv(csv_file, low_memory=False)
    
    df["date"] = pd.to_datetime(df["DATE"])
    df["HourlyPrecipitation"] = (df["HourlyPrecipitation"]
        .replace("T", "0.005")  # Replace 'T' (trace) with a small float
        .str.extract(r"([\d\.]+)")  # Extract numeric part, ignore non-numeric
        .astype(float)  # Convert to float
    )

    columns = ["date", "HourlyPrecipitation", "HourlyWindSpeed"]
    df = df[columns]
    
    df = df.dropna(subset=["date"])
    df.fillna(0, inplace=True)
    df.columns = df.columns.str.lower()

    return df

In [72]:
def clean_month_weather_data_daily(csv_file):
    df = pd.read_csv(csv_file, low_memory=False)
    df["date"] = pd.to_datetime(df["DATE"])

    df["DailyPrecipitation"] = (df["DailyPrecipitation"]
            .replace("T", "0.005")  
            .str.extract(r"([\d\.]+)")  
            .astype(float)  
    )
    df["DailySnowfall"] = (df["DailySnowfall"]
            .replace("T", "0.005")  
            .str.extract(r"([\d\.]+)")  
            .astype(float) 
    )
    df["DailySnowDepth"] = (df["DailySnowDepth"].replace("T", "0.005").astype(float))
    
    columns = ["date", "DailyPrecipitation", "DailyAverageWindSpeed", "DailyPeakWindSpeed",
               "DailySnowfall", "DailySnowDepth"]
    df = df[columns]

    df = df.dropna(subset=["date", "DailyPrecipitation", "DailyAverageWindSpeed", "DailyPeakWindSpeed",
               "DailySnowfall", "DailySnowDepth"])
    rename_map = {
        "date": "observation_date",
        "DailyPrecipitation": "daily_precipitation",
        "DailyAverageWindSpeed": "daily_average_wind_speed",
        "DailyPeakWindSpeed": "daily_peak_wind_speed",
        "DailySnowfall": "daily_snowfall",
        "DailySnowDepth": "daily_snow_depth",
    }
    df = df.rename(columns=rename_map)

    return df

In [73]:
def load_and_clean_weather_data():
    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [74]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [75]:
hourly_weather_data.head()

,date,hourlyprecipitation,hourlywindspeed
0,2020-01-01 00:51:00,0.0,8.0
1,2020-01-01 01:51:00,0.0,8.0
2,2020-01-01 02:51:00,0.0,14.0
3,2020-01-01 03:51:00,0.0,11.0
4,2020-01-01 04:51:00,0.0,6.0


In [76]:
hourly_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56098 entries, 0 to 11638
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 56098 non-null  datetime64[ns]
 1   hourlyprecipitation  56098 non-null  float64       
 2   hourlywindspeed      56098 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.7 MB


In [77]:
hourly_weather_data.describe()

,date,hourlyprecipitation,hourlywindspeed
count,56098,56098.000000,56098.000000
mean,2022-05-29 21:14:19.618881024,0.010841,4.537238
min,2020-01-01 00:51:00,0.000000,0.000000
25%,2021-03-18 19:01:45,0.000000,0.000000
50%,2022-05-28 01:21:00,0.000000,5.000000
75%,2023-08-15 05:39:00,0.000000,7.000000
max,2024-10-22 18:51:00,3.470000,2237.000000
std,NaN,0.056735,13.883208


In [78]:
daily_weather_data.tail(10)

,observation_date,daily_precipitation,daily_average_wind_speed,daily_peak_wind_speed,daily_snowfall,daily_snow_depth
11335,2022-12-22 23:59:00,0.23,8.6,36.0,0.000,0.0
11389,2022-12-23 23:59:00,1.83,10.0,45.0,0.005,0.0
11414,2022-12-24 23:59:00,0.00,11.3,33.0,0.000,0.0
11439,2022-12-25 23:59:00,0.00,8.5,31.0,0.000,0.0
11464,2022-12-26 23:59:00,0.00,7.1,20.0,0.000,0.0
11489,2022-12-27 23:59:00,0.00,5.5,17.0,0.000,0.0
11516,2022-12-28 23:59:00,0.00,5.1,26.0,0.000,0.0
11541,2022-12-29 23:59:00,0.00,6.1,21.0,0.000,0.0
11566,2022-12-30 23:59:00,0.00,2.9,16.0,0.000,0.0
11637,2022-12-31 23:59:00,0.28,1.8,16.0,0.000,0.0


In [79]:
daily_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1685 entries, 24 to 11637
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   observation_date          1685 non-null   datetime64[ns]
 1   daily_precipitation       1685 non-null   float64       
 2   daily_average_wind_speed  1685 non-null   float64       
 3   daily_peak_wind_speed     1685 non-null   object        
 4   daily_snowfall            1685 non-null   float64       
 5   daily_snow_depth          1685 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 92.1+ KB


In [80]:
daily_weather_data.describe()

,observation_date,daily_precipitation,daily_average_wind_speed,daily_snowfall,daily_snow_depth
count,1685,1685.000000,1685.000000,1685.000000,1685.000000
mean,2022-06-13 09:20:28.308605184,0.145490,5.008131,0.040890,0.160861
min,2020-01-01 23:59:00,0.000000,0.600000,0.000000,0.000000
25%,2021-04-08 23:59:00,0.000000,3.200000,0.000000,0.000000
50%,2022-06-21 23:59:00,0.000000,4.700000,0.000000,0.000000
75%,2023-08-17 23:59:00,0.060000,6.400000,0.000000,0.000000
max,2024-10-21 23:59:00,7.130000,14.200000,14.800000,14.000000
std,NaN,0.421219,2.339557,0.503529,1.062031


## Part 2: Storing Cleaned Data

In [105]:
engine = db.create_engine(DATABASE_URL)

In [106]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather(
    id INTEGER PRIMARY KEY,
    date DATETIME,
    hourly_precipitation FLOAT,
    hourly_wind_speed FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather(
    id INTEGER PRIMARY KEY,
    date DATETIME,
    daily_precipitation FLOAT,
    daily_average_wind_speed FLOAT,
    daily_peak_wind_speed FLOAT
    daily_snowfall FLOAT,
    daily_snow_depth FLOAT
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips(
     tpep_pickup_datetime DATETIME, 
     tpep_dropoff_datetime DATETIME,
     trip_distance FLOAT, 
     latitude_pickup FLOAT, 
     longitude_pickup FLOAT, 
     latitude_dropoff FLOAT, 
     longitude_dropoff FLOAT, 
     fare_amount FLOAT, 
     extra FLOAT, 
     mta_tax FLOAT, 
     tip_amount FLOAT, 
     tolls_amount FLOAT, 
     improvement_surcharge FLOAT, 
     congestion_surcharge FLOAT, 
     airport_fee FLOAT, 
     total_amount FLOAT,
     total_money FLOAT
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips(
    request_datetime DATETIME,
    on_scene_datetime DATETIME,
    pickup_datetime DATETIME,
    dropoff_datetime DATETIME,
    pickup_lat FLOAT,
    pickup_lon FLOAT,
    dropoff_lat FLOAT,
    dropoff_lon FLOAT,
    trip_miles FLOAT,
    total_fare FLOAT,
    tips FLOAT
);
"""

In [107]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [108]:
# create the tables with the schema files
with engine.connect() as connection:
    with open(DATABASE_SCHEMA_FILE, "r") as f:
        schema_sql = f.read()
    schema_stmts = [stmt.strip() for stmt in schema_sql.split(";") if stmt.strip()]
    for stmt in schema_stmts:
        connection.execute(db.text(stmt))

### Add Data to Database

In [109]:
def write_dataframes_to_table(table_to_df_dict):
    with engine.connect() as connection:
        for table_name, df in table_to_df_dict.items():
            
            df.to_sql(table_name, con=connection, if_exists='replace', index=False)

In [110]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather_data,
    "daily_weather": daily_weather_data,
}

In [111]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [117]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    query_filepath = os.path.join(QUERY_DIRECTORY, outfile)
    with open(query_filepath, 'w') as f:
        f.write(query)

### Query 1

In [118]:
QUERY_1_FILENAME = "taxi_most_popular_hour.sql"

QUERY_1 = """
SELECT 
    strftime('%H', tpep_pickup_datetime) AS hour,
    COUNT(*) AS ride_count
FROM 
    taxi_trips
WHERE 
    tpep_pickup_datetime BETWEEN '2020-01-01' AND '2024-08-31'
GROUP BY 
    hour
ORDER BY 
    ride_count DESC;
"""

In [119]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_1)).fetchall()
results

# or via pandas
df_results1 = pd.read_sql(QUERY_1, con=engine)

In [120]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

In [121]:
print(df_results1)

   hour  ride_count
0    17        1501
1    18        1455
2    15        1450
3    16        1352
4    19        1316
5    13        1271
6    14        1269
7    12        1229
8    11        1150
9    20        1079
10   10        1074
11   21         987
12   09         908
13   22         891
14   08         785
15   23         751
16   07         606
17   00         506
18   06         350
19   01         326
20   02         212
21   03         148
22   05         128
23   04         112


### Query 2

In [122]:
QUERY_2_FILENAME = "uber_most_popular_day.sql"

QUERY_2 = """
SELECT 
    strftime('%w', pickup_datetime) AS day_of_week,  -- 0 = Sunday, 6 = Saturday
    COUNT(*) AS ride_count
FROM 
    uber_trips
WHERE 
    pickup_datetime BETWEEN '2020-01-01' AND '2024-08-31'
GROUP BY 
    day_of_week
ORDER BY 
    ride_count DESC;
"""

In [123]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_2)).fetchall()
results

# or via pandas
df_results2 = pd.read_sql(QUERY_2, con=engine)

In [124]:
write_query_to_file(QUERY_2, QUERY_2_FILENAME)

In [125]:
print(df_results2)

  day_of_week  ride_count
0           6        3372
1           5        3287
2           4        3009
3           0        2926
4           3        2915
5           2        2697
6           1        2425


### Query 3

In [126]:
QUERY_3_FILENAME = "jan_per_distance.sql"

QUERY_3 = """
SELECT 
    trip_distance
FROM (
    SELECT 
        CAST(trip_distance AS FLOAT) AS trip_distance
    FROM taxi_trips 
    WHERE tpep_pickup_datetime BETWEEN '2024-01-01' AND '2024-01-31'
    
    UNION ALL
    
    SELECT 
        CAST(trip_miles AS FLOAT) AS trip_distance
    FROM uber_trips 
    WHERE pickup_datetime BETWEEN '2024-01-01' AND '2024-01-31'
) AS combined_results
WHERE trip_distance IS NOT NULL
ORDER BY trip_distance
LIMIT 1 OFFSET (
    SELECT CAST(COUNT(*) * 0.95 AS INTEGER) 
    FROM (
        SELECT 
            CAST(trip_distance AS FLOAT) AS trip_distance
        FROM taxi_trips 
        WHERE tpep_pickup_datetime BETWEEN '2024-01-01' AND '2024-01-31'
        
        UNION ALL
        
        SELECT 
            CAST(trip_miles AS FLOAT) AS trip_distance
        FROM uber_trips 
        WHERE pickup_datetime BETWEEN '2024-01-01' AND '2024-01-31'
    )
) - 1;
"""


In [127]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_3)).fetchall()
results

# or via pandas
df_results3 = pd.read_sql(QUERY_3, con=engine)

In [128]:
write_query_to_file(QUERY_3, QUERY_3_FILENAME)

In [129]:
print(df_results3)

   trip_distance
0          13.45


### Query 4

In [130]:
QUERY_4_FILENAME = "top10_busiest_day.sql"

QUERY_4 = """
WITH combined_rides AS (
    SELECT 
        DATE(tpep_pickup_datetime) AS trip_date,
        COUNT(*) AS total_rides,
        AVG(trip_distance) AS avg_distance
    FROM taxi_trips
    WHERE tpep_pickup_datetime BETWEEN '2023-01-01' AND '2023-12-31'
    GROUP BY trip_date

    UNION ALL

    SELECT 
        DATE(pickup_datetime) AS trip_date,
        COUNT(*) AS total_rides,
        AVG(trip_miles) AS avg_distance
    FROM uber_trips
    WHERE pickup_datetime BETWEEN '2023-01-01' AND '2023-12-31'
    GROUP BY trip_date
),
daily_stats AS (
    SELECT
        trip_date,
        SUM(total_rides) AS total_rides,
        AVG(avg_distance) AS avg_distance
    FROM combined_rides
    GROUP BY trip_date
)
SELECT 
    ds.trip_date,
    ds.total_rides,
    ds.avg_distance,
    dw.daily_precipitation AS avg_precipitation,
    dw.daily_average_wind_speed AS avg_wind_speed
FROM daily_stats ds
LEFT JOIN daily_weather dw ON ds.trip_date = DATE(dw.date)
ORDER BY ds.total_rides DESC
LIMIT 10;
"""

In [131]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_4)).fetchall()
results

# or via pandas
df_results4 = pd.read_sql(QUERY_4, con=engine)

OperationalError: (sqlite3.OperationalError) no such column: dw.date
[SQL: 
WITH combined_rides AS (
    SELECT 
        DATE(tpep_pickup_datetime) AS trip_date,
        COUNT(*) AS total_rides,
        AVG(trip_distance) AS avg_distance
    FROM taxi_trips
    WHERE tpep_pickup_datetime BETWEEN '2023-01-01' AND '2023-12-31'
    GROUP BY trip_date

    UNION ALL

    SELECT 
        DATE(pickup_datetime) AS trip_date,
        COUNT(*) AS total_rides,
        AVG(trip_miles) AS avg_distance
    FROM uber_trips
    WHERE pickup_datetime BETWEEN '2023-01-01' AND '2023-12-31'
    GROUP BY trip_date
),
daily_stats AS (
    SELECT
        trip_date,
        SUM(total_rides) AS total_rides,
        AVG(avg_distance) AS avg_distance
    FROM combined_rides
    GROUP BY trip_date
)
SELECT 
    ds.trip_date,
    ds.total_rides,
    ds.avg_distance,
    dw.daily_precipitation AS avg_precipitation,
    dw.daily_average_wind_speed AS avg_wind_speed
FROM daily_stats ds
LEFT JOIN daily_weather dw ON 

In [ ]:
write_query_to_file(QUERY_4, QUERY_4_FILENAME)

In [ ]:
print(df_results4)

### Query 5

In [ ]:
QUERY_5_FILENAME = 

In [ ]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_5)).fetchall()
results

# or via pandas
df_results5 = pd.read_sql(QUERY_5, con=engine)

In [ ]:
write_query_to_file(QUERY_5, QUERY_5_FILENAME)

In [ ]:
print(df_results5)

### Query 6

In [ ]:
QUERY_6_FILENAME = 

In [ ]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_6)).fetchall()
results

# or via pandas
df_results6 = pd.read_sql(QUERY_6, con=engine)

In [ ]:
write_query_to_file(QUERY_6, QUERY_6_FILENAME)

In [ ]:
print(df_results6)

## Part 4: Visualizing the Data

### Visualization 1

In [107]:
# use a more descriptive name for your function
def taxi_popular_hour(dataframe):
    # Sort the data by the 'hour' column to ensure it's in order
    df_sorted = df.sort_values(by="hour")
    
    # Create a bar chart
    plt.figure(figsize=(12, 6))
    plt.bar(df_sorted["hour"], df_sorted["ride_count"], color="skyblue", edgecolor="black")
    
    # Add labels and title
    plt.xlabel("Hour", fontsize=12)
    plt.ylabel("Number of Rides", fontsize=12)
    plt.title("Number of Rides by Hour of the Day", fontsize=14)
    plt.xticks(df_sorted["hour"], fontsize=10)
    plt.yticks(fontsize=10)
    
    # Show the plot
    plt.tight_layout()
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)